In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("punto_de_pedido.xlsx")

In [ ]:
def column_with_desired_data(raw_dataframe, art, descrip, stock, vpd,a='',b='',c = '',d=''):

    result_df = raw_dataframe[[art,descrip,vpd,stock]]
    if a != '':
        result_df = result_df.rename(columns={art:a})
  
    if  b != '':
        result_df = result_df.rename(columns={descrip:b})

    if c != '':
        result_df = result_df.rename(columns={vpd:c})

    if d != '':
        result_df = result_df.rename(columns={stock:d})    
         
    return result_df

In [ ]:
def sorted_and_cumulative_df(df, column_to_sort, cum_name):
    df = df.sort_values(column_to_sort, ascending=False)
    df[cum_name] = df[column_to_sort].cumsum()
    return df   

In [ ]:
def vpd_weight(df, cum_column, weights_name):
    total_vpd = list(df[cum_column])[-1]
    df[weights_name] = df[cum_column].apply(lambda x: (x/total_vpd)*100)
    return df

In [ ]:
def compare_with_limits(x,a,b):
    if x <= a:
        return 'A'
    elif  x <= b and x > a:
        return 'B'
    else:
        return 'C'

In [ ]:
def abc_classification(df,weights_column,a=80, b=95, category_col_name = 'categoria'):
    df[category_col_name] = df[weights_column].apply(lambda x: compare_with_limits(x,a,b))
    return df

In [ ]:
def dataframe_dictionary(complete_dataframe, category_column):
    dict_df = {}
    dict_df['A'] = df[df[category_column] == "A"]
    dict_df['B'] = df[df[category_column] == "B"]
    dict_df['C'] = df[df[category_column] == "C"]
    return dict_df

In [ ]:
def stock_with_price(df1, column_df1,df2, column_df2,value_column, price_column_name,stock_column_name):
    df1 = df1.set_index(column_df1)
    df2 = df2.set_index(column_df2)
    result = pd.concat([df1, df2], axis=1, join="inner")
    result[value_column] = result[price_column_name] * result[stock_column_name]
    result[value_column] = result[value_column].apply(lambda x: int(x))
    return result

In [ ]:
df = column_with_desired_data(df,'ARTÍCULO','DESCRIPCIÓN','VTA. PROMEDIO','STOCK','articulo','descripcion','venta promedio','stock')

In [ ]:
df = sorted_and_cumulative_df(df,'venta promedio','acumulado VPD')

In [ ]:
vpd_weight(df,'acumulado VPD','Peso de VPD acum.')

In [ ]:
result_df = abc_classification(df,'Peso de VPD acum.')

In [ ]:
df_pc_raw = pd.read_excel('20230518130636_PRV-1_FECVIG-20230517_plantillaPreciosCompra.xlsx', header = 1)

In [ ]:
df_pc_filtered = df_pc_raw[['Artículo','Precio Neto + I. Internos']]


In [ ]:
stock_classified_with_value = stock_with_price(result_df,'articulo',df_pc_filtered,'Artículo','Valorizado','Precio Neto + I. Internos','stock')

In [ ]:
df_dict = dataframe_dictionary(result_df,'categoria')